<a href="https://colab.research.google.com/github/swang066/CR-for-Promoter/blob/main/model/CR_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import datasets, layers, models, Model
import matplotlib.pyplot as plt

from math import *
from tensorflow import keras

from tensorflow import keras
from keras.models import Model, load_model
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adam, SGD
from keras.layers import *
from keras.metrics import *
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from hyperopt import hp, STATUS_OK

In [ ]:
data = pd.read_csv("data/GSM4144998_DPRinVitro_rep1.txt",sep='\t')
score = data.score.to_numpy()
seq = data.Sequence.to_numpy()
dict = {
    'A': [1,0,0,0],
    'T': [0,1,0,0],
    'C': [0,0,1,0],
    'G': [0,0,0,1]
}
X = np.zeros([len(score),19,4])
for i in range(len(score)):
  tmp = seq[i]
  tmp = np.array([dict[char] for char in tmp])
  X[i] = tmp
scaler = StandardScaler()
Y = scaler.fit_transform(score.reshape(-1,1)).reshape(-1)
test_pick = pd.read_csv("data/test.txt",header=None).to_numpy() - 1
samples = pd.read_csv("data/samples.txt",header=None).to_numpy() - 1
XTest = X[test_pick.transpose()[0]]
YTest = Y[test_pick.transpose()[0]]
X = np.delete(X,test_pick.transpose()[0],0)
Y = np.delete(Y,test_pick.transpose()[0],0)
train_size = 200000
XTrain = X[samples[range(int(train_size*0.9))].transpose()[0]]
XVal = X[samples[range(int(train_size*0.9),train_size)].transpose()[0]]
YTrain = Y[samples[range(int(train_size*0.9))].transpose()[0]]
YVal = Y[samples[range(int(train_size*0.9),train_size)].transpose()[0]]

In [ ]:
layer_0 = Input(shape=X.shape[1:],name='Input')

layer_1 = Conv1D(32,4, kernel_initializer='glorot_normal',activation='relu',name='Conv1D_1')(layer_0)
layer_2 = MaxPool1D(2,name='MaxPool_1')(layer_1)

layer_3 = Conv1D(16,7,padding='same', kernel_initializer='glorot_normal',activation='relu',name='Conv1D_2')(layer_2)
layer_4 = MaxPool1D(2,name='MaxPool_2')(layer_3)

layer_5 = Conv1D(8,7,padding='same', kernel_initializer='glorot_normal',activation='relu',name='Conv1D_3')(layer_4)
layer_6 = MaxPool1D(2,name='MaxPool_3')(layer_5)

layer_7 = Flatten(name='Flatten_1')(layer_6)

layer_8 = Dense(16,name='Dense_1')(layer_7)
layer_9 = ReLU(name='ReLU_1')(layer_8)
layer_10 = Dropout(0.00099,name='dropout_1')(layer_9)

layer_11 = Dense(2,name='Dense_2')(layer_10)
layer_12 = ReLU(name='ReLU_2')(layer_11)
layer_13 = Dropout(0.00099,name='dropout_2')(layer_12)

layer_14 = Dense(19*4,kernel_regularizer=tf.keras.regularizers.l1(0.0001),name='Contextual_Weight')(layer_13)
layer_15 = Flatten(name='Flatten_2')(layer_0)
layer_16 = Multiply(name='Multiply')([layer_14,layer_15])

layer_17 = Dense(1,kernel_initializer='ones',use_bias=False,name='Sum')(layer_16)
layer_18 = Dense(1,name='Output')(layer_17)
model = Model(inputs=layer_0,outputs=layer_18)
model.layers[17].trainable = False
model.summary()
keras.utils.plot_model(model, "model_with_shape_info.png", show_shapes=True)

In [ ]:
model.compile(SGD(lr=0.0005, momentum=0.9),'mean_squared_error', metrics=['mean_squared_error'])
earlystop_cb = EarlyStopping(monitor='val_loss', patience=15, verbose=1, mode='min')
check_cb = ModelCheckpoint('bestparams.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
result = model.fit(XTrain, YTrain, batch_size=128, shuffle="batch", epochs=100,
        validation_data=[XVal, YVal], callbacks=[earlystop_cb, check_cb])
mse_history = result.history['val_mean_squared_error']

In [ ]:
best_file = 'bestparams/CR-3_bestparams.h5'
model = load_model(best_file)
YTrain_Pred = model.predict(XTrain)
print('Correlation of training dataset: '+str(np.corrcoef(YTrain[:].transpose(),YTrain_Pred.transpose())[0,1]))
YVal_Pred = model.predict(XVal)
print('Correlation of training dataset: '+str(np.corrcoef(YVal[:].transpose(),YVal_Pred.transpose())[0,1]))
YTest_Pred = model.predict(XTest)
print('Correlation of training dataset: '+str(np.corrcoef(YTest[:].transpose(),YTest_Pred.transpose())[0,1]))